In [1]:
# read data 
import numpy as np
pdt_ta=[]
with open('./hotel.txt', 'r') as f:
    lines=f.readlines()
    for li in lines:
        li=li.strip()
        if li:
            vals=li.split(' ')
            vals=[float(val) for val in vals]
            pdt_ta.append(vals)
print(len(pdt_ta))
print(len(pdt_ta[0]))
pdt_ta=np.array(pdt_ta)        

user_ta=[]
with open('./user.txt', 'r') as f:
    lines=f.readlines()
    for li in lines:
        li=li.strip()
        if li:
            vals=li.split(' ')
            vals=[float(val) for val in vals]
            user_ta.append(vals)
print(len(user_ta))
print(len(user_ta[0]))
user_ta=np.array(user_ta)           

1850
7
137563
7


In [13]:
# MDA+
import numpy as np

def dominate(v1, v2):
    for i,j in zip(v1, v2):
        if i<j:
            return False
    return True

def kskyband(data, k):
    do_cnt=np.array([0 for _ in range(len(data))])
    ret=[]
    for i in range(len(data)):
        for j in range(i+1, len(data)):
            if do_cnt[i]>=k:
                break
            if dominate(data[i], data[j]):
                do_cnt[j]+=1
            elif dominate(data[j], data[i]):
                do_cnt[i]+=1
        if do_cnt[i]<k:
            ret.append(i)
    return ret

In [14]:
# option dataset to kskyband
k=15
print(len(pdt_ta))
kskyid=kskyband(pdt_ta, k)
pdt_ksky=pdt_ta[kskyid, :]


1850


In [4]:
from itertools import combinations
import heapq

def closeTo(d):
    id_=int(d);
    idp1=int(id_+1)
    idm1=int(id_-1);
    d1=abs(d-idm1);
    d2=abs(d-id_);
    d3=abs(d-idp1);
    if d1>d2:
        if d3>d2:
            return id_;
        else:
            return idp1;
    else:
        if d3>d1:
            return idm1;
        else:
            return idp1;

def rdominate(v1, v2):
    assert(len(v1)==len(v2))
    for i, j in zip(v1, v2):
        if v2>v1:
            return False
    return True
    
class cell:
    
    def __init__(self,b, cur_level, tar_level, K):
        self.bounds=b
        self.dim=int(len(b)/2)
        self.uHeat=0.0
        self.rHeat=0.0
        self.dHeat=0.0
        self.cur_l=cur_level
        self.tar_l=tar_level
        self.k=K
        self.get_vertexes()
        self.heap=[0.0 for _ in range(K)]
        self.mdap_s_rksyband=[]
        self.s_rskyband=[]
        self.rkskyband=[]
        self.rskyband_lb_MDA=[]

    
    def get_vertexes(self):
        sum_lb=0.0
        for d in range(self.dim):
            sum_lb=self.bounds[d*2]
        dis=self.bounds[1]-self.bounds[0]
        num_ub=closeTo((1.0-sum_lb)/dis)
        for cb in combinations(range(dim), num_ub):
            v=[0 for _ in range(self.dim)]
            for i in cb:
                v[i]=1
            one_cb=[]
            for i in range(self.dim):
                if v[i]==1:
                    one_cb.append(self.bounds[2*i+1])
                else:
                    one_cb.append(self.bounds[2*i])
            self.verrtexes.append(one_cb)
    
    def get_lb_ub(self, p):
        lb=1e9;
        ub=0.0;
        for v in self.vertexes:
            tmp=np.dot(p, v);
            if tmp>ub:
                ub=tmp
            if tmp<lb:
                lb=tmp
        return lb, ub
    
    def get_scores(self, ps, P, lb, ub):
        lb=[0.0 for _ in range(len(ps))]
        ub=[0.0 for _ in range(len(ps))]
        for i in range(len(ps)):
            self.get_lb_ub(P[ps[i]], lb[i], ub[i])
            
    def isLeaf(self):
        return cur_l>=tar_l
    
    def get_next_children(it):
        u=1.0
        dis=(self.bounds[1]-self.bounds[0])/2  # child should be divided by 2
        l=1.0-dis*self.dim
        child_p_num=(1<<dim)
        while it<child_p_num:
            lv= [0.0 for i in range(dim)]
            iter_cp=it
            for j in range(dim):
                if iter_cp%2==0:
                    lv[j]=self.bounds[j*2]
                else:
                    lv[j]=(self.bounds[j*2]+self.bounds[j*2+1])/2.0
                iter_cp>>=1
            slv=sum(lv)
            if u>slv and slv>1.0:
                child_b=[0.0 for _  in range(dim*2)]
                for i in range(dim):
                    child_b[i*2]=lv[i];
                    child_b[i*2+1]=lv[i]+dis;
                return cell(child_b, self.cur_l+1, self.tar_l, self.k), it
            else:
                ++it;
        return None, it
    
    def MDA_superSet2RKS(self, P):
        theta=self.heap[0];
        tmp=[]
        for i in s_rskyband:
            ub=i[2]
            if ub>theta:
                tmp.append(i)
        tmp=sorted(tmp, key=lambda item: item[2], reverse=True)
        scores=[]
        for i in range(k):
            self.rkskyband.append(tmp[i][0])
            self.rskyband_lb_MDA.append(tmp[i][1])
            s=[]
            for v in self.vertexes:
                s.append(np.dot(v, P[tmp[i][0]]))
            scores.append(s)
        for i in range(k, len(tmp)):
            r_dominate_count=0
            s=[]
            for j in range(len(self.rkskyband)):
                if tmp[i][1] < self.rskyband_lb_MDA[j]:
                    if len(s)==0:
                        for v in self.vertexes:
                            s.append(np.dot(v, P[tmp[i][0]]))
                    if r_dominate(scores[j], s):
                        r_dominate_count+=1
                        if r_dominate_count>=self.k:
                            break
            if r_dominate_count<k:
                self.dHeat+=r_dominate_count
                self.rkskyband.append(tmp[i][0])
                self.rskyband_lb_MDA.append(tmp[i][1])
                if len(s)==0:
                    for v in self.vertexes:
                        s.append(np.dot(v, P[tmp[i][0]]))
                scores.append(s)
 
    
    def MDAp_insert(self, parent_sRSK, P, ret):
        lbs=[]
        ubs=[]
        self.get_scores(parent_sRSK, P, lbs, ubs)
        for lb in lbs:
            theta=self.heap[0]
            if lb>theta:
                heappush(self.heap, lb) # heap push
                heappop(self.heap)
        theta=self.heap[0]
        for i in range(parent_sRSK):
            if ubs[i]>theta:
                this.mdap_s_rksyband.push_back(parent_sRSK[i])
                if self.isLeaf():
                    self.s_rskyband.append([parent_sRSK[i], lbs[i]], ubs[i])
        if cur_l<tar_l:
            child_p_num=1<<self.dim
            i=0
            while i<child_p_num:
                child, i=self.get_next_children(i)
                if child is not None:
                    child.MDAp_insert(self.mdap_s_rksyband, P, ret)
        if self.isLeaf():
            self.MDA_superSet2RKS(P)
        
def MDAp(root, data, ret):
    idx=[i for i in range(len(data))]
    root.MDAp_insert(idx, data, ret)
    
    

In [ ]:
    def __init__(self,b, cur_level, tar_level, K):
b=[]
for j in range(dim):
    b.append(0.0)
    b.append(1.0)